In [1]:
!pip install tensorflow transformers flask networkx matplotlib torch


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Replace this placeholder with your own model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Replace this function to process motion data (could be sensor or image data)
def predict_motion(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1)
    return "Correct Motion" if predictions.item() == 1 else "Incorrect Motion"


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Replace this placeholder with your own model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Replace this function to process motion data (could be sensor or image data)
def predict_motion(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1)
    return "Correct Motion" if predictions.item() == 1 else "Incorrect Motion"


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from flask import Flask, request, jsonify
import networkx as nx

app = Flask(__name__)

# Define a simple knowledge graph for pain management (you can expand it)
G = nx.Graph()
G.add_node("Fitness Motion", type="exercise")
G.add_node("Shoulder Stretch", type="exercise")
G.add_node("Neck Rotation", type="exercise")
G.add_node("Pain Relief", type="treatment")
G.add_node("Physical Therapy", type="treatment")
G.add_edge("Fitness Motion", "Shoulder Stretch")
G.add_edge("Fitness Motion", "Neck Rotation")
G.add_edge("Shoulder Stretch", "Pain Relief")
G.add_edge("Neck Rotation", "Physical Therapy")

# Function to retrieve pain management recommendations
def get_pain_management_recommendations(prediction):
    recommendations = []
    if prediction == "Correct Motion":
        recommendations = [n for n in G.neighbors("Shoulder Stretch") if G.nodes[n]['type'] == 'treatment']
    elif prediction == "Incorrect Motion":
        recommendations = [n for n in G.neighbors("Neck Rotation") if G.nodes[n]['type'] == 'treatment']
    return recommendations

# Route for motion detection and pain management recommendation
@app.route('/predict', methods=['POST'])
def predict():
    text = request.json.get('text')  # This is where motion data should be input
    prediction = predict_motion(text)

    # Get pain management recommendations based on the prediction
    recommendations = get_pain_management_recommendations(prediction)

    # Return prediction and recommendations as JSON
    return jsonify({
        "prediction": prediction,
        "recommendations": recommendations
    })

# Run the Flask server
if __name__ == "__main__":
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [5]:
from flask import Flask, request, jsonify
import networkx as nx
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load your pre-trained motion detection model (replace with your model)
model_name = "bert-base-uncased"  # Placeholder
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Flask setup
app = Flask(__name__)

# Placeholder knowledge graph for pain management
G = nx.Graph()
G.add_node("Fitness Motion", type="exercise")
G.add_node("Shoulder Stretch", type="exercise")
G.add_node("Neck Rotation", type="exercise")
G.add_node("Pain Relief", type="treatment")
G.add_node("Physical Therapy", type="treatment")
G.add_edge("Fitness Motion", "Shoulder Stretch")
G.add_edge("Fitness Motion", "Neck Rotation")
G.add_edge("Shoulder Stretch", "Pain Relief")
G.add_edge("Neck Rotation", "Physical Therapy")

# Motion detection function
def predict_motion(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1)
    return "Correct Motion" if predictions.item() == 1 else "Incorrect Motion"

# Pain management recommendations from knowledge graph
def get_pain_management_recommendations(prediction):
    recommendations = []
    if prediction == "Correct Motion":
        recommendations = [n for n in G.neighbors("Shoulder Stretch") if G.nodes[n]['type'] == 'treatment']
    elif prediction == "Incorrect Motion":
        recommendations = [n for n in G.neighbors("Neck Rotation") if G.nodes[n]['type'] == 'treatment']
    return recommendations

# Flask route for predictions
@app.route('/predict', methods=['POST'])
def predict():
    text = request.json.get('text')
    prediction = predict_motion(text)
    pain_management_recommendations = get_pain_management_recommendations(prediction)
    return jsonify({
        "prediction": prediction,
        "recommendations": pain_management_recommendations
    })

# Running the Flask app
if __name__ == "__main__":
    app.run()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [6]:
# from pyngrok import ngrok

# # Start Flask app in the background
# !nohup flask run --host=0.0.0.0 --port=5000 &

# # Expose the Flask app using ngrok
# public_url = ngrok.connect(5000)
# print(f"Public URL: {public_url}")

In [7]:
pip install pyngrok

In [8]:
# from pyngrok import ngrok

# # Run Flask in the background
# !FLASK_APP=app.py flask run --host=0.0.0.0 --port=5000 &

# # Expose the Flask app to the public
# public_url = ngrok.connect(5000)
# print(f"Public URL: {public_url}")


In [ ]:
import requests

url = f'{public_url}/predict'
data = {"text": "The participant performed a shoulder rotation exercise with correct posture."}

response = requests.post(url, json=data)
print(response.json())


In [12]:
import pickle

# Load the bicep model
with open('/content/bicep_dp.pkl', 'rb') as f:
    bicep_model = pickle.load(f)

# Load the squats model
with open('/content/LR_model.pkl', 'rb') as f:
    squats_model = pickle.load(f)


In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load and preprocess motion detection data
df = pd.read_csv('/content/train.csv')
df.head()

,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_knee_z,right_knee_v,left_ankle_x,left_ankle_y,left_ankle_z,left_ankle_v,right_ankle_x,right_ankle_y,right_ankle_z,right_ankle_v
0,down,0.600248,0.433268,-0.006637,0.999949,0.650759,0.517787,0.041632,0.994121,0.571711,...,-0.133012,0.976462,0.659328,0.836607,-0.294549,0.994219,0.575143,0.814380,-0.157936,0.966270
1,down,0.600160,0.449894,-0.008615,0.999966,0.651127,0.535333,0.036240,0.994432,0.571456,...,-0.115595,0.974204,0.659568,0.831231,-0.279601,0.993431,0.576560,0.812989,-0.135235,0.959162
2,down,0.599425,0.466234,-0.047636,0.999957,0.651464,0.550511,-0.021722,0.994265,0.571824,...,-0.084128,0.949108,0.659044,0.828410,-0.202085,0.987354,0.575949,0.813362,-0.053549,0.928765
3,down,0.597382,0.485779,-0.057994,0.999926,0.651386,0.571049,-0.023268,0.994848,0.572379,...,-0.069531,0.921725,0.658455,0.832319,-0.168358,0.982599,0.576511,0.813286,-0.058735,0.904242
4,down,0.594474,0.438586,-0.041261,0.999931,0.650657,0.518908,-0.003251,0.993209,0.571241,...,-0.119639,0.969358,0.658565,0.836644,-0.274596,0.994046,0.576144,0.815093,-0.117950,0.965770


In [16]:
y.unique()

array(['down', 'up'], dtype=object)

In [25]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the array
y = label_encoder.fit_transform(y)

In [23]:
X = df.drop('label', axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Neural Network model
nn_model = Sequential([
    Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')
])
nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# nn_model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Ensemble: Majority Voting
rf_preds = rf_model.predict(X_test_scaled)
nn_preds = np.argmax(nn_model.predict(X_test_scaled), axis=1)
# ensemble_preds = np.round((rf_preds + nn_preds) / 2)

# Evaluation
# print(f"Ensemble Model Accuracy: {accuracy_score(y_test, ensemble_preds)}")
# print(classification_report(y_test, ensemble_preds))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [31]:
from tensorflow.keras.layers import Input

# Neural Network model with Input layer
nn_model = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),  # Define input shape
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')
])
nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
accuracy = nn_model.evaluate(X_test_scaled, y_test)
# Train the model
# nn_model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2)
print(accuracy)

0.9
